In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import concurrent.futures

In [10]:
df_before = pd.read_csv('~/Documents/GitHub/2022-Pergi-Kuliner-Restaurant-Data/pergikuliner_restaurants_dataset.csv')
df_before.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Momo Paradise,Tipe KulinerJepang,"\n Mal Kelapa Gading 5, Lanta...",https://pergikuliner.com/restaurants/jakarta/m...,Cabang: ya,"""Tunai,Visa,Master,Debet""",<bound method PageElement.get_text of <div cla...,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"100% Halal,Ruang VIP,Bisa reservasi,Area parki...","Wifi,Pesan antar,Area merokok,Area outdoor,24 ...",021 45853709,4.71,"4.9,4.5,4.9,4.7,4.6"
1,Caspar,Tipe KulinerSpanyol,\n Sudirman Suites Apartment ...,https://pergikuliner.com/restaurants/jakarta/c...,"PembayaranTunai, Debet","""Tunai,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Area outdoor,100% Halal,Bisa...","Pesan antar,24 Jam,Ruang VIP,Vegetarian",082217817880,4.58,"4.7,4.8,4.8,4.7"
2,Furusato Izakaya,Tipe KulinerJepang,\n Sudirman Suites Apartment ...,https://pergikuliner.com/restaurants/jakarta/f...,"PembayaranTunai, Visa, Master, Debet","""Tunai,Visa,Master,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Area outdoor,Ruang VIP,Bisa ...","Pesan antar,24 Jam,100% Halal,Vegetarian",08119250020,4.47,"4.5,4.8,4.0,4.6,4.5"
3,Osteria Gia,Tipe KulinerItalia,"\n Pacific Place Mall, Lantai...",https://pergikuliner.com/restaurants/jakarta/o...,Cabang: ya,"""Tunai,Visa,Master,Debet""",NaN,\n Di atas Rp. 200.000 /orang...,<bound method PageElement.get_text of <time it...,"Wifi,Area merokok,Bisa reservasi,Area parkir,A...","Pesan antar,Area outdoor,24 Jam,100% Halal,Rua...",021 5153300,4.45,"4.6,4.5,4.1,4.5,4.5"
4,Isshin,Tipe KulinerJepang,"\n Central Park, Lantai Lower...",https://pergikuliner.com/restaurants/jakarta/i...,Cabang: ya,"""Tunai,Visa,Master,Debet,Go-pay""",NaN,\n Rp. 100.000 - Rp. 200.000 ...,<bound method PageElement.get_text of <time it...,"Wifi,100% Halal,Bisa reservasi,Area parkir","Pesan antar,Area merokok,Area outdoor,24 Jam,R...",021 56985157,4.46,"4.7,4.4,4.1,4.7,4.5"


In [ ]:
#cleared
#col 1, 2, 3, 9, 10 and 12 do not have nans.
#col 4: fixed branch data does not exist and the columns got filled with other information. 
#col 5: cleared. look precise enough, removed the quotation marks.
#col 11: fixed 57 phone number nans. some of these are just not scraped.
#col 13: fixed bot does not scrape rating when it goes below 4.
#col4: fixed put 'kapasitas kursi' in the right cols.

#awaiting script rerun
#col 6: promotional texts does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.
#col 8: schedule does not get scraped correctly. added str() to get_text() method but the script needs to be rerun.

In [14]:
def retrieve_schpromo(ulink, browser):
    browserDriver = browser
    browserDriver.get(ulink)
    soup = BeautifulSoup(browserDriver.page_source)

    properties = {'itemprop': 'openingHours',
                  'class': 'promotional-text'}
    subbucket = []
    for key, val in properties.items():
        results = soup.find(attrs={key: val}).get_text() if not None else None
        subbucket.append(results)
    subbucket = pd.Series(subbucket)

    return subbucket

browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")

bucket = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    f = executor.submit(retrieve_schpromo, df_before['3'], browser)
    bucket.append(f.result())

browser.close()

df = pd.concat(bucket)

/tmp/ipykernel_19732/1528229698.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")


TypeError: cannot concatenate object of type '<class 'method'>'; only Series and DataFrame objs are valid

In [128]:
# fixed put 'kapasitas kursi' in the right cols.
pd.set_option('max_colwidth', 120)
df['14'] = df_before.loc[df_before['4'].str.contains('Kapasitas'), '4']

In [103]:
#fixed branch data does not exist and the columns got filled with other information
df['4'] = df['4'].str.contains('Cabang: ya')

In [60]:
# fixed 57 phone number nans. some of these are just not scraped.
def retrieve_phone(ulink):
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
    browser.get(ulink)
    soup = BeautifulSoup(browser.page_source)
    phone = str(soup.find(href=re.compile('tel:')).get_text())
    browser.close()
    return phone

missing_phone_url = df.loc[df['11'].isna(), '3']

for nums in missing_phone_url.index:
    missing_phone_url[nums] = retrieve_phone(missing_phone_url[nums])

df.loc[df['11'].isna(), '11'] = missing_phone_url

/tmp/ipykernel_3735/2660351575.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")


In [90]:
# fixed bot does not scrape rating when it goes below 4.
def retrieve_ratings(ulink):
    browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")
    browser.get(ulink)
    soup = BeautifulSoup(browser.page_source)
    phone = ','.join([i.getText().strip() for i in soup.find_all(class_='rate-box-bottom')])
    browser.close()
    return phone

missing_ratings = df.loc[df.iloc[:,13].str.len() < 19, '13']

for idx in missing_ratings.index:
    missing_ratings[idx] = retrieve_ratings(df.loc[idx, '3'])

df.loc[df.iloc[:,13].str.len() < 19, '13'] = missing_ratings

/tmp/ipykernel_3735/3001592676.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver")


In [97]:
#checks
df['13'].str.len().value_counts()

19    1500
Name: 13, dtype: int64

In [64]:
#checks
missing_phone_url

53        021 5753274
54       021 29880942
62        021 7209966
68       081287620750
72        021 7209966
76       021 29938888
78        022 2030333
79       021 31921111
96       021 29399588
117      021 22771888
126      021 23583896
155       021 5747777
163      021 25501993
164      021 62201900
168       031 5458888
177      021 55911777
183      021 80630888
212       031 7310010
228       031 5468000
286       031 2880388
306      021 29880888
328      021 27887788
385      021 29361010
426       022 2041873
454       031 5477488
466      021 22771888
507      021 29921383
531      021 23583898
548     021 2295 6067
554      021 27887768
566      021 25518888
567      021 39507355
596      021 29229999
663      021 29200088
665       021 3805555
692     021 2295 6067
698      021 27887788
754       021 6692093
791      021 39703339
804      021 29938888
805      021 31936868
847      031 60038888
850      021 73910050
851      031 99251222
908      021 25981666
939       

In [36]:
#checks
df['5'].str.replace('"', '').str.split(',', expand=True).stack().value_counts()

Tunai     1480
Debet     1370
Visa      1225
Master    1225
Ovo        395
Go-pay     376
Dana       221
dtype: int64

In [38]:
df['4'].value_counts()

Cabang: ya                                                 966
PembayaranTunai, Visa, Master, Debet                       255
PembayaranTunai                                             39
PembayaranTunai, Visa, Master, Debet, Go-pay, Ovo, Dana     36
PembayaranTunai, Debet                                      32
PembayaranTunai, Visa, Master, Debet, Go-pay, Ovo           32
PembayaranTunai, Debet, Go-pay, Ovo, Dana                   13
Kapasitas:  50 kursi                                        12
PembayaranTunai, Visa, Master, Debet, Ovo                   11
Kapasitas:  30 kursi                                         7
PembayaranTunai, Visa, Master, Debet, Go-pay                 6
PembayaranTunai, Debet, Go-pay, Ovo                          6
Kapasitas:  100 kursi                                        5
PembayaranTunai, Go-pay, Ovo, Dana                           5
Kapasitas:  70 kursi                                         5
Kapasitas:  20 kursi                                   

In [37]:
# fixed
df['5'] = df['5'].str.replace('"', '')

In [23]:
['Name', 'culinary_type', 'address', 'url', '']

['Name', 'culinary_type', 'address', 'url', '']